### Inputs

In [1]:
# PAPERMILL (Not yet implemented)
project = 'MeridianHills'
model_name = 'F01'
book = 'Uniform'

wse_file = 's3://pfra/RiskAssessment/{0}/Results/{1}/WSE_{0}_{1}_{2}.csv'.format(project, model_name, book)
breach_prob_file = 's3://pfra/RiskAssessment/{0}/BreachAnalysis/{0}_{1}_raw_prob_table.csv'.format(project, model_name)
weights_file = 's3://pfra/RiskAssessment/{0}/BreachAnalysis/{0}_{1}.xlsx'.format(project, model_name)
output_file = 's3://pfra/RiskAssessment/{0}/Results/{1}/AAL_{0}_{1}_{2}.csv'.format( project, model_name,book)

structure_cols = {'Unique Building ID' : 'plus_code',
             'Damage Code': 'damage_code',
             'Building Limit' : 'bldg_limit',
             'Building Deduction' : 'bldg_ded',
             'Content Limit' : 'cnt_limit',
             'Content Deduction': 'cnt_ded',
             'Ground Elevation': 'GroundElev',
             'First Floor Height': 'first_floor_elev'}

In [2]:
import sys; sys.path.append('../core')
from risk_refactor import *
import time
from multiprocessing import Pool, cpu_count
print('Working Directory = {}'.format(os.getcwd()))

Working Directory = C:\Users\Administrator\Desktop\probmod-tools\risk


### Adjust curves for current FEMA implementation

In [3]:
defaultHazusDDFn_path = '../risk/hazusdepthdmgfns/Building_DDF_Full_LUT_Hazus3p0.json'
df_BDDFn = pd.read_json(defaultHazusDDFn_path, orient = 'index')
df_BDDFn = hazusID_to_depth(df_BDDFn)
df_agg = aggregate_ddf_curves(df_BDDFn, curve_groups, plot=False)

# set curve for single family 1 story no basement
df_agg.loc[-1, 'singFam_1Story_NoBasement']= 0.6
df_agg.loc[-0.000000000001, 'singFam_1Story_NoBasement']= 0.6

# set curve for single family 2 story no basement
df_agg.loc[-1, 'singFam_2Story_NoBasement']= 0.6
df_agg.loc[-0.000000000001, 'singFam_2Story_NoBasement']= 0.6

# set curve for single family 3 story no basement
df_agg.loc[-1, 'singFam_3Story_NoBasement']= 0.0
df_agg.loc[-0.000000000001, 'singFam_3Story_NoBasement']= 0.0

# set curve for mobile home
df_agg.loc[-0.000000000001, 'mobileHome']= 0.75

df_agg = df_agg.sort_index()

### Read in all of the relevant data

In [5]:
dfwse = pd.read_csv(wse_file, index_col='plus_code')
dfw = pd.read_excel(weights_file, sheet_name='Event_Weights', index_col=0 )[['Overall Weight']]
dfbp = pd.read_csv(breach_prob_file, sep='\t', index_col=0)
for col in dfbp.columns:
    dfbp.rename(columns={col:col.split('_')[1]}, inplace=True)

### Pack up all arguments for multiprocessing

In [7]:
events_data_object = FluvialEvents(dfw, dfbp)
allargs = [(p_code, events_data_object, df_agg, dfwse, dfw, dfbp, structure_cols) for p_code in dfwse.index]

### Compute AALs

In [8]:
st = time.time()

p = Pool(cpu_count())
results = p.map(calc_aal_mp, allargs)
p.close()
aal_results = {k: v for k, v in results}

print(round((time.time()-st)/60, 2), 'minutes to process')

24.85 minutes to process


In [9]:
df_AAL_only = pd.DataFrame.from_dict([aal_results]).T.rename(columns={0:'AAL'})
df_AAL_only.index.name = structure_cols['Unique Building ID']
# df_AAL_only.to_csv(output_file)
df_AAL_only.to_csv('AAL_{}_{}_{}.csv'.format(project, model_name, book))

### Check out the results

In [28]:
from IPython.display import display, Markdown
tot = df_AAL_only.AAL.sum()
mx = df_AAL_only.AAL.max()
avg = df_AAL_only.AAL.mean()
display(Markdown('## Sum: ${0:,.2f}'.format(tot)))
display(Markdown('## Max: ${0:,.2f}'.format(mx)))
display(Markdown('## Mean: ${0:,.2f}'.format(avg)))

## Sum: $12,290,164.72

## Max: $55,359.61

## Mean: $1,078.56

# END